## Objetivos da aula
- Extrair dados públicos de compras governamentais
- Tratar os dados
- Armazenar os dados em um arquivo

## Extração

In [30]:
! pip install pyspark

In [31]:
import requests
import io
import pandas as pd
from pyspark.sql import SparkSession

In [32]:
# puxando dados da api do governo
url = 'http://compras.dados.gov.br/servicos/v1/servicos.csv?descricao=educação'
data = requests.get(url).content
# por default ele precisa retornar status code 200 para dar certo
data

b'C\xc3\xb3digo,Descri\xc3\xa7\xc3\xa3o,Unidade medida,CPC,Se\xc3\xa7\xc3\xa3o,Divis\xc3\xa3o,Grupo,Classe,Subclasse\n14311,"Orienta\xc3\xa7\xc3\xa3o / Educa\xc3\xa7\xc3\xa3o - Atividade F\xc3\xadsica                                                                                ",UNHOMEM/H M\xc3\x8aS,9659,"9: SERVI\xc3\x87OS DA COMUNIDADE, PESSOAIS E SOCIAIS E OUTROS SERVI\xc3\x87OS                                                            ","96: SERVI\xc3\x87OS RECREACIONAIS, CULTURAIS E ESPORTIVOS                                                                          ","965: SERVI\xc3\x87OS RELACIONADOS COM ESPORTES E SERVI\xc3\x87OS RECREACIONAIS DO ESPORTE                                                  ","9659: OUTROS ESPORTES E SERVI\xc3\x87OS DE ESPORTES RECREATIVOS                                                                      ",\n15946,"Servi\xc3\xa7o Penitenci\xc3\xa1rio - Reeduca\xc3\xa7\xc3\xa3o e Reabilita\xc3\xa7\xc3\xa3o de Apenado                             

In [33]:
# converter os dados de bytes para objeto
parse_Data = io.StringIO(data.decode("UTF-8"))
parse_Data

In [34]:
# criando dataframe
df = pd.read_csv(parse_Data)
df.head()

,Código,Descrição,Unidade medida,CPC,Seção,Divisão,Grupo,Classe,Subclasse
0,14311,Orientação / Educação - Atividade Física ...,UNHOMEM/H MÊS,9659,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...","96: SERVIÇOS RECREACIONAIS, CULTURAIS E ESPORT...",965: SERVIÇOS RELACIONADOS COM ESPORTES E SERV...,9659: OUTROS ESPORTES E SERVIÇOS DE ESPORTES R...,NaN
1,15946,Serviço Penitenciário - Reeducação e Reabilita...,UN,979,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",97: OUTROS SERVIÇOS ...,979: OUTROS SERVIÇOS DIVERSOS/MISCELÂNEA ...,NaN,NaN
2,18481,Consultoria e Assessoria - Educação ...,UN,8319,"8: SERVIÇOS DE ARQUITETURA, DESENHOS TÉCNICOS,...","83: OUTROS SERVIÇOS DE NEGÓCIOS, TÉCNICOS E PR...",831: SERVIÇOS DE CONSULTORIA E DE GERÊNCIA/GES...,"8319: OUTROS SERVIÇOS DE GERÊNCIA/GESTÃO, EXCE...",NaN
3,19321,Curso / Treinamento Educação - Distância ...,UN,929,"9: SERVIÇOS DA COMUNIDADE, PESSOAIS E SOCIAIS ...",92: SERVIÇOS DA INSTRUÇÃO/ EDUCAÇÃO/ ENSINO ...,929: OUTROS SERVIÇOS DE EDUCAÇÃO E TREINAMENTO...,NaN,NaN


In [35]:
spark = SparkSession.builder.appName("PipelineDadosXP").getOrCreate()

In [36]:
type(df)

pandas.core.frame.DataFrame

In [37]:
# transformar o df de Pandas para Spark
df = spark.createDataFrame(df)

In [38]:
type(df)

pyspark.sql.dataframe.DataFrame

## Transformação

In [39]:
df.printSchema()

root
 |-- Código: long (nullable = true)
 |-- Descrição: string (nullable = true)
 |-- Unidade medida: string (nullable = true)
 |-- CPC: long (nullable = true)
 |-- Seção: string (nullable = true)
 |-- Divisão: string (nullable = true)
 |-- Grupo: string (nullable = true)
 |-- Classe: string (nullable = true)
 |-- Subclasse: double (nullable = true)



In [40]:
df.show(10)

+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
|Código|           Descrição|Unidade medida| CPC|               Seção|             Divisão|               Grupo|              Classe|Subclasse|
+------+--------------------+--------------+----+--------------------+--------------------+--------------------+--------------------+---------+
| 14311|Orientação / Educ...| UNHOMEM/H MÊS|9659|9: SERVIÇOS DA CO...|96: SERVIÇOS RECR...|965: SERVIÇOS REL...|9659: OUTROS ESPO...|      NaN|
| 15946|Serviço Penitenci...|            UN| 979|9: SERVIÇOS DA CO...|97: OUTROS SERVIÇ...|979: OUTROS SERVI...|                 NaN|      NaN|
| 18481|Consultoria e Ass...|            UN|8319|8: SERVIÇOS DE AR...|83: OUTROS SERVIÇ...|831: SERVIÇOS DE ...|8319: OUTROS SERV...|      NaN|
| 19321|Curso / Treinamen...|            UN| 929|9: SERVIÇOS DA CO...|92: SERVIÇOS DA I...|929: OUTROS SERVI...|                 NaN|   

In [41]:
df.columns

['Código',
 'Descrição',
 'Unidade medida',
 'CPC',
 'Seção',
 'Divisão',
 'Grupo',
 'Classe',
 'Subclasse']

In [42]:
# metodo para renomear colunas do df
df = (df.withColumnRenamed('Código', 'Codigo')
      .withColumnRenamed('Descrição', 'Descricao')
      .withColumnRenamed('Unidade medida', 'UnidadeMedida')
      .withColumnRenamed('Seção', 'Secao')
      .withColumnRenamed('Divisão', 'Divisao'))

df.columns

['Codigo',
 'Descricao',
 'UnidadeMedida',
 'CPC',
 'Secao',
 'Divisao',
 'Grupo',
 'Classe',
 'Subclasse']

In [45]:
# utilizando compreensão de listas para colocar todas as letras das colunas em minusculo
df = df.select([x.lower() for x in df.columns])
df.printSchema()

root
 |-- codigo: long (nullable = true)
 |-- descricao: string (nullable = true)
 |-- unidademedida: string (nullable = true)
 |-- cpc: long (nullable = true)
 |-- secao: string (nullable = true)
 |-- divisao: string (nullable = true)
 |-- grupo: string (nullable = true)
 |-- classe: string (nullable = true)
 |-- subclasse: double (nullable = true)



## Carga

In [46]:
from google.colab import drive

In [48]:
# para salvar as alterações no Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
dir = '/content/drive/MyDrive/XPE/data'

In [52]:
df.write.mode("overwrite")\
        .csv(f"{dir}/compras.csv")

In [53]:
print(spark.read.csv("/content/drive/MyDrive/XPE/data/compras.csv").show())

+-----+--------------------+-------------+----+--------------------+--------------------+--------------------+--------------------+---+
|  _c0|                 _c1|          _c2| _c3|                 _c4|                 _c5|                 _c6|                 _c7|_c8|
+-----+--------------------+-------------+----+--------------------+--------------------+--------------------+--------------------+---+
|18481|Consultoria e Ass...|           UN|8319|8: SERVIÇOS DE AR...|83: OUTROS SERVIÇ...|831: SERVIÇOS DE ...|8319: OUTROS SERV...|NaN|
|19321|Curso / Treinamen...|           UN| 929|9: SERVIÇOS DA CO...|92: SERVIÇOS DA I...|929: OUTROS SERVI...|                 NaN|NaN|
|14311|Orientação / Educ...|UNHOMEM/H MÊS|9659|9: SERVIÇOS DA CO...|96: SERVIÇOS RECR...|965: SERVIÇOS REL...|9659: OUTROS ESPO...|NaN|
|15946|Serviço Penitenci...|           UN| 979|9: SERVIÇOS DA CO...| 97: OUTROS SERVIÇOS|979: OUTROS SERVI...|                 NaN|NaN|
+-----+--------------------+-------------+----+-